In [1]:
from sklearn import datasets, model_selection
data = datasets.load_iris()
Xtrain, Xtest, ytrain, ytest = model_selection.train_test_split(data['data'], data['target']) 

In [2]:
import torch
from torch import nn, optim

In [3]:
import os
def resume_from_checkpoint(path_to_checkpoint):

  if os.path.isfile(path_to_checkpoint):

    # Caricamento del checkpoint
    checkpoint = torch.load(path_to_checkpoint)

    # Ripristino dello stato del sistema
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    opt.load_state_dict(checkpoint['opt'])
    print("Caricato il checkpoint '{}' (epoca {})"
                  .format(path_to_checkpoint, checkpoint['epoch']))

  else:
    start_epoch = 0

  return start_epoch

In [4]:
Xtrain = torch.from_numpy(Xtrain).float()
Xtest = torch.from_numpy(Xtest).float()
ytrain = torch.from_numpy(ytrain)
ytest = torch.from_numpy(ytest)

In [5]:
from torch.utils import data
train_data = data.TensorDataset(Xtrain, ytrain)

In [6]:
class CustomModel(nn.Module):

    def __init__(self):
        super(CustomModel, self).__init__()
        
        self.hidden = nn.Linear(4, 10)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(0.2)
        self.out = nn.Linear(10, 3)
        
    def forward(self, x):
        x = self.relu(self.hidden(x))
        return self.out(self.drop(x))

In [7]:
net = CustomModel()

In [8]:
print(net)

CustomModel(
  (hidden): Linear(in_features=4, out_features=10, bias=True)
  (relu): ReLU()
  (drop): Dropout(p=0.2, inplace=False)
  (out): Linear(in_features=10, out_features=3, bias=True)
)


In [9]:
loss = nn.CrossEntropyLoss()
opt = torch.optim.Adam(params=net.parameters(), lr=0.01)

In [10]:
train_data_loader = data.DataLoader(train_data, batch_size=32, shuffle=True)

In [11]:
def train_step(x, y):

  # Modalità di training
  net.train()

  # Calcola le predizioni
  y_pred = net(x)

  # Calcola funzione costo
  loss_epoch = loss(y_pred, y)

  # Esegui back-propagation
  loss_epoch.backward()

  # Aggiorna i parametri
  opt.step()

  # Resetta il gradiente
  opt.zero_grad()

for epoch in range(100):
  net.train()
  for Xb, yb in train_data_loader:
    train_step(Xb, yb)

In [12]:
with torch.no_grad():
  net.eval()
  y_pred = net(Xtest)
  correct = (y_pred.max(dim=1)[1] == ytest)
  print(torch.mean(correct.float()).item())

0.9210526347160339


# Checkpoints

In [13]:
torch.save(net.state_dict(), './tmp')

In [14]:
net.load_state_dict(torch.load('./tmp'))

<All keys matched successfully>

In [15]:
start_epoch = resume_from_checkpoint('checkpoint.pth.tar')
for epoch in range(start_epoch, 1000):

  net.train()

  for Xb, yb in train_data_loader:  
    train_step(Xb, yb)

  # Stato complessivo del processo di ottimizzazione
  state = {
    'epoch': epoch,
    'state_dict': net.state_dict(),
    'opt': opt.state_dict(),
  }
  torch.save(state, 'checkpoint.pth.tar') 

# CUDA

In [ ]:
torch.cuda.get_device_capability(0)

In [ ]:
net = CustomModel()
if torch.cuda.is_available():
  net.to('cuda')
loss = nn.CrossEntropyLoss()
opt = torch.optim.Adam(params=net.parameters(), lr=0.01)

In [ ]:
for epoch in range(2500):
  net.train()
  for Xb, yb in train_data_loader:
    Xb, yb = Xb.to(device), yb.to(device)
    train_step(Xb, yb)